In [1]:
using Distributed
addprocs(4);

In [2]:
using Pkg; Pkg.status()

Project pulse_input_DDM v0.0.1
    Status `~/Projects/inProgress/pulse_input_DDM.jl/Project.toml`
  [717857b8] DSP v0.5.2
  [31c24e10] Distributions v0.18.0 ⚲
  [f6369f11] ForwardDiff v0.10.3
  [033835bb] JLD2 v0.1.2
  [d3d80556] LineSearches v7.0.1
  [23992714] MAT v0.5.0+ #v0.7-update (https://github.com/halleysfifthinc/MAT.jl)
  [429524aa] Optim v0.18.1
  [eadc2687] Pandas v1.3.0
  [d330b81b] PyPlot v2.8.1
  [276daf66] SpecialFunctions v0.7.2
  [2913bbd2] StatsBase v0.30.0
  [ade2ca70] Dates 
  [8ba89e20] Distributed 
  [37e2e46d] LinearAlgebra 
  [9a3f8284] Random 


In [ ]:
@everywhere

In [3]:
@everywhere using pulse_input_DDM

CompositeException: On worker 2:
ArgumentError: Package pulse_input_DDM not found in current path:
- Run `import Pkg; Pkg.add("pulse_input_DDM")` to install the pulse_input_DDM package.

require at ./loading.jl:823
eval at ./boot.jl:319
#116 at /usr/local/miniconda/conda-bld/julia_1554006467847/work/usr/share/julia/stdlib/v1.0/Distributed/src/process_messages.jl:276
run_work_thunk at /usr/local/miniconda/conda-bld/julia_1554006467847/work/usr/share/julia/stdlib/v1.0/Distributed/src/process_messages.jl:56
run_work_thunk at /usr/local/miniconda/conda-bld/julia_1554006467847/work/usr/share/julia/stdlib/v1.0/Distributed/src/process_messages.jl:65
#102 at ./task.jl:259
#remotecall_wait#154(::Base.Iterators.Pairs{Union{},Union{},Tuple{},NamedTuple{(),Tuple{}}}, ::Function, ::Function, ::Distributed.Worker, ::Module, ::Vararg{Any,N} where N) at /usr/local/miniconda/conda-bld/julia_1554006467847/work/usr/share/julia/stdlib/v1.0/Distributed/src/remotecall.jl:421
remotecall_wait(::Function, ::Di

In [8]:
#define how many trials to generate data from and the time binning scale (in seconds)
nsessions = 10
ntrials = 200 * nsessions;

In [9]:
#parameters of the latent model
pz_gen = Dict("generative" => vcat(1.,13.,-0.5,10.,1.,0.4,0.02), 
    "name" => vcat("σ_i","B", "λ", "σ_a","σ_s","ϕ","τ_ϕ"),
    "fit" => vcat(trues(7)),
    "initial" => vcat(2.,15.,-5.,100.,2.,0.2,0.005),
    "lb" => [eps(), 4., -5., eps(), eps(), eps(), eps()],
    "ub" => [10., 100, 5., 800., 40., 2., 10.])

N,dimy = 3 * nsessions,3
#parameters for the neural observation model
py_gen = Dict("generative" => repeat([vcat(0., 1., 0.)],outer=N), 
    "name" => repeat(["neuron"],outer=N),
    "fit" => repeat([trues(dimy)],outer=N),
    "initial" => repeat([Vector{Float64}(undef,dimy)],outer=N));

In [12]:
data = pulse_input_DDM.sample_clicks(Int(ntrials/nsessions))

MethodError: MethodError: no method matching sample_clicks(::Int64)
Closest candidates are:
  sample_clicks(::Int64, !Matched::Float64) at /Users/briandepasquale/.julia/packages/pulse_input_DDM/5H4Vm/src/sample_model_functions.jl:29

In [ ]:
 
    
    data["spike_times"] = pmap((T,L,R,N,rng) -> sample_spikes_single_trial(pz,py,T,L,R;
        f_str=f_str, rng=rng, dtMC=dtMC), data["T"], data["leftbups"], data["rightbups"],
        shuffle(1:length(data["T"])))    

In [ ]:
    

function sample_spikes_single_trial(pz, py, T::Float64, L::Vector{Float64}, R::Vector{Float64};
         f_str::String="softplus", dtMC::Float64=1e-4, rng::Int=1,
         λ0::Vector{Float64}=Vector{Float64}())
    
    Random.seed!(rng)    
    #removed 4/30, and moved towards unbinned spike times instead, then a binning procedure after this
    #A = decimate(sample_latent(T,L,R,pz;dt=dtMC), Int(dt/dtMC))   
    A = sample_latent(T,L,R,pz;dt=dtMC)
    Y = map(py-> poisson_noise.(fy22(py, A, λ0, f_str=f_str), dtMC), py)    
    #add a find here
    
end

In [18]:
#generate some simulated clicks times and trial durations
data = sample_input_and_spikes_multiple_sessions(pz, py, N, nsessions, ntrials)

In [8]:
#compute the likelihood of the data, given the generative parameters
compute_LL(pz_gen[:generative], py_gen[:generative],data)

-7582.192527173623

In [9]:
#find the ML parameters with gradient descent
@time pz_gen, py_gen, = optimize_model(pz_gen, py_gen, data);

┌ Warning: some parameter(s) at lower bound. bumped it (them) up 1/4 from the lower bound.
└ @ pulse_input_DDM /mnt/bucket/people/briandd/Projects/pulse_input_DDM.jl/src/wrapper_functions.jl:60


Iter     Function value   Gradient norm 
     0     9.687366e+03     3.198976e+03
     1     7.792058e+03     4.743642e+02
     2     7.704790e+03     3.820503e+02
     3     7.699942e+03     9.942071e+01
     4     7.649685e+03     2.560999e+02
     5     7.635002e+03     3.324201e+02
     6     7.632300e+03     3.420436e+02
     7     7.606781e+03     2.445243e+02
     8     7.604548e+03     2.450761e+02
     9     7.599977e+03     1.302866e+02
    10     7.586483e+03     7.070546e+01
    11     7.584141e+03     6.693907e+00
    12     7.583859e+03     5.636726e+00
    13     7.583497e+03     4.571295e+00
    14     7.583272e+03     1.739027e+01
    15     7.582921e+03     1.829216e+01
    16     7.582691e+03     2.900603e+01
    17     7.582486e+03     8.659761e+00
    18     7.582398e+03     6.109909e+00
    19     7.582221e+03     6.334875e+00
    20     7.581995e+03     8.112058e+00
    21     7.580931e+03     7.603607e+00
    22     7.580896e+03     7.392573e+00
    23     7.580

   210     7.576103e+03     2.413856e-04
   211     7.576103e+03     8.312162e-04
   212     7.576103e+03     9.448297e-04
   213     7.576103e+03     1.868885e-03
   214     7.576103e+03     1.181312e-03
   215     7.576103e+03     3.332367e-04
   216     7.576103e+03     2.707114e-04
   217     7.576103e+03     1.336396e-03
   218     7.576103e+03     1.808745e-03
   219     7.576103e+03     1.866897e-03
   220     7.576103e+03     3.897698e-03
   221     7.576103e+03     4.838802e-03
   222     7.576103e+03     1.249398e-02
   223     7.576103e+03     1.001219e-02
   224     7.576103e+03     7.813503e-03
   225     7.576103e+03     7.123233e-03
   226     7.576103e+03     7.282269e-03
   227     7.576103e+03     7.518316e-03
   228     7.576103e+03     7.623064e-03
   229     7.576103e+03     7.606059e-03
   230     7.576103e+03     5.273775e-03
   231     7.576103e+03     1.181034e-03
   232     7.576103e+03     1.114812e-03
   233     7.576103e+03     6.346012e-04
   234     7.576

([8.70362e-15, 18.1487, -0.217549, 0.000106316, 1.1691, 0.313634, 0.0170549], [0.0583064, 0.096892], Results of Optimization Algorithm
 * Algorithm: BFGS
 * Starting Point: [0.6931471805599452,2.3978952727983707, ...]
 * Minimizer: [-32.40088013188025,2.649623770841773, ...]
 * Minimum: 7.576103e+03
 * Iterations: 378
 * Convergence: true
   * |x - x'| ≤ 1.0e-16: true 
     |x - x'| = 2.78e-17 
   * |f(x) - f(x')| ≤ 1.0e-16 |f(x)|: true
     |f(x) - f(x')| = 0.00e+00 |f(x)|
   * |g(x)| ≤ 1.0e-04: false 
     |g(x)| = 3.11e-04 
   * Stopped by an increasing objective: false
   * Reached Maximum Number of Iterations: false
 * Objective Calls: 682
 * Gradient Calls: 379, Optim.BFGSState{Array{Float64,1},Array{Float64,2},Float64,Array{Float64,1}}([-32.4009, 2.64962, -0.0435373, -9.14909, 0.156235, -1.15953, -4.07132, 0.0583064, -1.11612], [-32.4009, 2.64962, -0.0435373, -9.14909, 0.156235, -1.15953, -4.07132, 0.0583064, -1.11612], [1.0015e-9, 0.000123315, -0.000149779, -1.20259e-5, -0.0003

In [10]:
#compute the Hessian of the LL landscape, to compute confidence intervals on the parameters
pz_gen, py_gen = compute_H_CI!(pz_gen, py_gen, data);

(7×7 DataFrames.DataFrame. Omitted printing of 1 columns
│ Row │ generative │ name   │ fit  │ initial │ final       │ CI_plus   │
│     │ Float64    │ String │ Bool │ Float64 │ Float64     │ Float64   │
├─────┼────────────┼────────┼──────┼─────────┼─────────────┼───────────┤
│ 1   │ 1.0        │ σ_i    │ true │ 2.0     │ 8.70362e-15 │ Inf       │
│ 2   │ 13.0       │ B      │ true │ 15.0    │ 18.1487     │ 143.911   │
│ 3   │ -0.5       │ λ      │ true │ -2.5    │ -0.217549   │ 0.0849904 │
│ 4   │ 10.0       │ σ_a    │ true │ 100.0   │ 0.000106316 │ Inf       │
│ 5   │ 1.0        │ σ_s    │ true │ 2.0     │ 1.1691      │ 1.82322   │
│ 6   │ 0.4        │ ϕ      │ true │ 0.2     │ 0.313634    │ 0.532436  │
│ 7   │ 0.02       │ τ_ϕ    │ true │ 0.005   │ 0.0170549   │ 0.0247319 │, 2×7 DataFrames.DataFrame. Omitted printing of 1 columns
│ Row │ generative │ name   │ fit  │ initial │ final     │ CI_plus  │
│     │ Float64    │ String │ Bool │ Float64 │ Float64   │ Float64  │
├─────┼─────────

In [11]:
#identify which ML parameters have generative parameters within the CI 
pz_gen[:within_bounds] = (pz_gen[:CI_minus] .< pz_gen[:generative]) .& (pz_gen[:CI_plus] .> pz_gen[:generative])
pd_gen[:within_bounds] = (pd_gen[:CI_minus] .< pd_gen[:generative]) .& (pd_gen[:CI_plus] .> pd_gen[:generative]);

In [12]:
using DataFrames
show(DataFrames.DataFrame(pz_gen),allcols=true)
show(DataFrames.DataFrame(py_gen),allcols=true)

,generative,name,fit,initial,final,CI_plus,CI_minus,within_bounds
,Float64,String,Bool,Float64,Float64,Float64,Float64,Bool
1,1.0,σ_i,true,2.0,8.70362e-15,Inf,2.22045e-16,true
2,13.0,B,true,15.0,18.1487,143.911,5.43081,true
3,-0.5,λ,true,-2.5,-0.217549,0.0849904,-0.518501,true
4,10.0,σ_a,true,100.0,0.000106316,Inf,2.22045e-16,true
5,1.0,σ_s,true,2.0,1.1691,1.82322,0.74966,true
6,0.4,ϕ,true,0.2,0.313634,0.532436,0.184748,true
7,0.02,τ_ϕ,true,0.005,0.0170549,0.0247319,0.011761,true


In [14]:
#sample data from the model using the ML parameters
sampled_choices = pmap((T,L,R) -> sample_model(T,L,R,pz_gen[:final],pd_gen[:final]),
    data["T"],data["leftbups"],data["rightbups"])

#compute the final click difference, which will dictate the correct choice
ΔLR = map((nT,L,R)->diffLR(nT,L,R,dt),data["nT"],data["leftbups"],data["rightbups"])
ΔLR = map(x-> x[end], ΔLR);